## OBE simulation data explorer

In [1]:
import scipy.io as sio
import glob2
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
import fproc as fp
%matplotlib inline

In [2]:
foldr = r'R:\My Code\OBESim_Apu'
Apus, matlist = fp.SortNamesBy(glob2.glob(foldr + r'\*0.mat'), 'Apu=(\d+)')

# Combine simulations at different pump amplitudes
dimvec = np.hstack((sio.loadmat(matlist[0])['PES'].shape, len(matlist)))
matstack = np.zeros(dimvec)
for i, mat in enumerate(matlist):
    matstack[:,:,:,:,:,i] = sio.loadmat(mat)['PES']
matstack.shape

(61L, 81L, 9L, 8L, 7L, 8L)

In [26]:
import ipywidgets as wg
from IPython.display import display
from ipywidgets import interact

# Pump amplitude (A_pu) slider
Slider_Apu = wg.IntSlider(
    value=11,
    min=1,
    max=15,
    step=2,
    description=r'\(A_{pu}\)'+' (a.u.)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
Slider_Apu.style.handle_color = 'lightblue'

# Pump detuning (Del_a) slider
Slider_Dela = wg.IntSlider(
    value=-10,
    min=-10,
    max=6,
    step=2,
    description=r'\(\Delta_{a} (\times \Delta_E) \)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
Slider_Dela.style.handle_color = 'lightblue'

# Intermediate state relaxation rate (Gamma_nn) slider
Slider_Gnn = wg.IntSlider(
    value=5,
    min=5,
    max=33,
    step=4,
    description=r'\(\Gamma_{nn}^{-1}\)'+' (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
Slider_Gnn.style.handle_color = 'lightblue'

# Intermediate state dephasing rate (Gamma_in) slider
Slider_Gin = wg.IntSlider(
    value=5,
    min=5,
    max=95,
    step=15,
    description=r'\(\Gamma_{in}^{-1}\)'+' (fs)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='i',
    slider_color='white'
)
Slider_Gin.style.handle_color = 'lightblue'
    
def browseImgStack(imgmat):

    tdlay = np.arange(-50e-15, 101e-15, 2.5e-15)/1e-15
    Delb = np.arange(-20, 20.1, 0.5)
    Dela_vec = np.arange(-10, 7, 2)
    Gin_vec = np.arange(5, 96, 15)
    Gnn_vec = np.arange(5,34,4)
    Apu_vec = Apus
    
    def plot_image2d(Dela, Gnn, Gin, Apu):
        fig, ax = plt.subplots(1, 1)
        nDela = np.where(Dela_vec == Slider_Dela.value)[0][0]
        nGnn = np.where(Gnn_vec == Slider_Gnn.value)[0][0]
        nGin = np.where(Gin_vec == Slider_Gin.value)[0][0]
        nApu = np.where(Apu_vec == Slider_Apu.value)[0][0]
        x, y = np.meshgrid(tdlay, Delb)
        plt.pcolormesh(x, y, imgmat[:,:,nDela,nGnn,nGin,nApu].T, cmap='terrain')
        ax.set_aspect('auto')
        ax.set_xlim([min(tdlay), max(tdlay)])
        ax.set_ylim([min(Delb), max(Delb)])
        ax.set_xlabel('Probe delay (fs)', fontsize=15)
        ax.set_ylabel('Probe energy detuning', fontsize=15)
        plt.tight_layout()
        plt.show()

    interact(plot_image2d, Dela=Slider_Dela, Gnn=Slider_Gnn, Gin=Slider_Gin, Apu=Slider_Apu)

In [27]:
browseImgStack(matstack)